In [1]:
import tensorflow as tf
import random
import time

/home/ktai17/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.set_random_seed(777)

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
'''
1. First Layer
1-1. Conv
'''

'\n1. First Layer\n1-1. Conv\n'

In [6]:
learning_rate = 0.001
training_epochs = 15  # training 하는 단위.
batch_size = 100      # 한 번에 몇개씩 가져오는가.

In [7]:
X = tf.placeholder(tf.float32, [None, 28*28])
Y = tf.placeholder(tf.float32, [None, 10])

X_img = tf.reshape(X, [-1, 28, 28, 1])

In [8]:
'''
간단한 function으로 해결해보자!
'''

'\n간단한 function으로 해결해보자!\n'

In [9]:
# Convolutional Layer #1
conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)

In [10]:
# Pooling layer #1
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], padding="SAME", strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=True)

In [11]:
# Convolutional Layer #2
conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)

In [12]:
# Pooling layer #2
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], padding="SAME", strides=2)
dropout2= tf.layers.dropout(inputs=pool2, rate=0.7, training=True)

In [13]:
# Convolutional Layer #3
conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)

In [14]:
# Pooling layer #3
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], padding="SAME", strides=2)
dropout3 = tf.layers.dropout(inputs=pool3, rate=0.7, training=True)

In [15]:
# Dense layer(Fully connected layer) with Relu #4 
# (128*4*4 -> 625 -> 10) 
flat=tf.reshape(dropout3, [-1, 4*4*128])        #X_img:28 -> layer1:14 -> layer2:7 ->layer3:3.5(rounding->4)
dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=True)

In [16]:
# Logits (no activation) Layer #5
# Layler 5 Final Fc 625 inputs -> 10 outputs

### Softmax

In [17]:
logits = tf.layers.dense(inputs=dropout4, units=10)
#activation function(relu)를 안쓰는 이유는 cost의 tf.nn.softmax에 softmax가 activation function으로 들어가있으므로!
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
### initialize
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [19]:
### training model
print('Learning started. It takes sometime...')
t1 = time.time()
for epoch in range(training_epochs):
    avg_cost=0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    print('Epoch:','%04d'%(epoch+1), 'cost=', '{:.9f}'.format(avg_cost))

print('Learning Finished!!!')
t2=time.time()

Learning started. It takes sometime...
Epoch: 0001 cost= 0.753265107
Epoch: 0002 cost= 0.289651331
Epoch: 0003 cost= 0.224066913
Epoch: 0004 cost= 0.197165402
Epoch: 0005 cost= 0.182579792
Epoch: 0006 cost= 0.172054565
Epoch: 0007 cost= 0.162309661
Epoch: 0008 cost= 0.151177028
Epoch: 0009 cost= 0.151854805
Epoch: 0010 cost= 0.146061593
Epoch: 0011 cost= 0.143056157
Epoch: 0012 cost= 0.142107383
Epoch: 0013 cost= 0.139654088
Epoch: 0014 cost= 0.135824401
Epoch: 0015 cost= 0.133702106
Learning Finished!!!


In [20]:
correction_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correction_prediction, tf.float32))

print('Accuracy: ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
print('Running Time: '+str(t2-t1))

Accuracy:  0.9607
Running Time: 337.88442850112915
